# Deploying your FastAPI

You have an API that is able to send back some JSON responses. Usually, an API is intended for other people to be used. Thus, the API you create should be deployed to a web server so other users can make requests to it. 

In this notebook we are going to show you two alternatives for deploying your FastAPI app: [EC2 instance](#ec2-instance) and [Deta](#deta)

# EC2 Instance

When running an application using uvicorn, you can check how it looks going to the localhost and port you specified, which by default are `127.0.0.1` and `8000` respectively. You can do something similar but allowing the rest of the world accessing your application if they connect to your EC2 instance and ir redirects the user to the API.

To deploy your API to an EC2 instance, follow these steps:
1. Create a GitHub repo, and upload your project
2. Create an Ubuntu 20.04 EC2 instance. Make sure you enable the HTTP, and SSH ports in the Inbound rules of the Security Group. Additionally, add a custom TCP (Transmission Control Protocol) connection that accepts port 8000 (or the one you want to use when lauching your app through uvicorn). You might be wondering why we need to establish the TCP rule if we are going to use ; essentially TCP is used to establish the connection by opening the port that you are going to use to launch your app. For example, in this case, we will use port 8000 to launch the FastAPI app.



<p align=center><img src=images/connections.png width=400></p>


3. SSH to your instance and update

```    
        sudo apt update
        sudo apt upgrade -y
```    

4. Install python dependencies: 

```        
        sudo apt-get install -y -q python3-pip python3-dev python3-venv
```

5. Create the work directory and create a venv:

```
        sudo mkdir /var/myapp
        cd /var/myapp
        sudo python3 -m venv venv
        . venv/bin/activate
```

6. Clone your repo into `src` and move inside:

```    
        sudo git clone <your repo URL> src
        cd src
```    

7. Install your packages:

```
        sudo pip install -r requirements.txt
```

8. Run your `main.py`. Make sure that, if you run it directly using python, main.py calls for uvicorn inside with the right port numbers. If you run it from uvicorn, you have to specify the ports in the command line. Thus, the last part of `main.py` can look like this:

In [ ]:
if __name__ == '__main__':
    configure_routing()
    uvicorn.run(app, port=8000, host='0.0.0.0')
else:
    configure_routing()

So you can call it either by running:

`sudo python3 main.py`

Or

`uvicorn --host 0.0.0.0 --port 8000 main:app`

You can check if it worked by going to the public IP of your EC2 instance, and add :8000. In this example, the public IP is `54.236.20.149`, so to check if it worked, I connected to http://54.236.20.149:8000/docs, and this is the result:

<p align=center><img src=images/EC2_Fastapi.png width=600></p>

And you can start making requests to it

In [3]:
import requests

requests.get('http://54.236.20.149:8000/api/dob/Steve?last_name=Carrel&city=true').json()

{'first name': 'Steve',
 'last_name': 'Carrel',
 'Date of Birth': '1962-08-16',
 'City': 'Concord, Massachusetts, U.S.'}

## Making your API scalable

One of the advantages of running your API on a virtual machine is that you can run your API using nginx and Gunicorn. In short, they are going to allow multiple users connect to your API at the same time

<p align=center><img src=images/Gunicorn.png width=400></p>

While nginx will handle the https calls, Gunicorn will fan out the calls to several Uvicorn servers. This structure is actually common in other web frameworks such as Flask and Django.

To get all these features implemented for your application, you can create a new EC2 instance without the custom connection and you can add a HTTPS connection rule in case you will add an SSL to the application

<p align=center><img src=images/connections_2.png width=400></p>

Then follow the same steps until step 6, and follow the steps below.

Alternatively, you can change the inbound rules of the existing EC2


7. Install your packages, nginx, and gunicorn
```
        sudo pip install -r requirements.txt
        sudo pip install gunicorn
        sudo apt-get install nginx
```
8. Right now, if we access the HTTP address corresponding to the EC2 instance, we will see the next webpage:<p align=center><img src=images/nginx.png width=600></p> which means that your EC2 is already hosting a nginx server. But you need to tell nginx what to listen to. Thus, you need to change the nginx configuration:
```    
        sudo vim /etc/nginx/sites-available/myapp
```


- Use the following settings:
```
            server{
                server_name <Public IPv4 address of your EC2>;
                location / {
                    include proxy_params;
                    proxy_pass http://127.0.0.1:8000;
                }
            }
```
The proxy_pass parameter uses the default Uvicorn settings, but if you are using other settings, make sure you change them in the settings above.



9. Create a soft link between the settings and the sites enabled by nginx: 

    - `sudo ln -s /etc/nginx/sites-available/myapp /etc/nginx/sites-enabled/`
10. Restart your nginx server: `sudo systemctl restart nginx.service` and you are ready to use gunicorn!
11. Before that, let's check everything works fine. Run `sudo python3 main.py`. If you go to the HTTP address of your console, and add `/docs` (for example `http://54.205.133.239/docs`), you should see something like this:
<p align=center><img src=images/nginx2.png width=1000></p>


12. This is great, but you are not still using running it using Gunicorn. You can run multiple uvicorn servers using the following command:
    - `gunicorn -w 4 -k uvicorn.workers.UvicornWorker main:app`

You will see that 4 workers have started:
<p align=center><img src=images/gunicorn_ec2.png width=400></p>




And the HTTP still works with no issue, in fact, it will accept more requests in parallel!

13. The final step is to leave gunicorn and nginx running on the background using a daemon. First deactivate your venv, and create a new .service file:
    - `deactivate`
    - `sudo vim /etc/systemd/system/myapp.service`

In the file, include:
```
[Unit]
Description=Gunicorn instance to serve MyApp
After=network.target

[Service]
User=ubuntu
Group=www-data
WorkingDirectory=/var/myapp/src
Environment="PATH=/var/myapp/venv/bin"
ExecStart=gunicorn -w 4 -k uvicorn.workers.UvicornWorker main:app

[Install]
WantedBy=multi-user.target
```


14. Finally, start this service file on the background: `sudo systemctl start myapp.service`

Now, everytime you go to that HTTP address, you will have access to the API!

If you want to add some security to it, you can look up how to add SSL to your website. But for now, feel proud that your API is up and running!

If you want a quicker solution, you can try to deploy your application using Deta.

# Deta

[Deta](https://docs.deta.sh/docs/home/) allows you to upload your applications to a cloud server. It is the recommended platform by the FastAPI documentation. Even though it is not very scalable:

- An execution times out after 10s. 
- 128 MB of RAM for each execution. 
- Read-only file system. Only /tmp can be written to. It has a 512 MB storage limit.
- Invocations have an execution processes/threads limit of 1024.
- HTTP Payload size limit is 5.5 MB.

It is a great way to test your applications

<p align=center><img src=images/Deta.png width=600></p>

Deploying your app to Deta takes just a few steps:
1. Sign Up in Deta
2. Installing the CLI: `curl -fsSL https://get.deta.dev/cli.sh | sh` or `iwr https://get.deta.dev/cli.ps1 -useb | iex` if you are on Windows (inside Powershell)
3. Login from your terminal using `deta login`
4. Your project should have (at least) the following structure:
```
root/
│
├── requirements.txt
└── main.py
```
and the name of the variable that stores your FastAPI object is named `app`:

In [ ]:
# main.py
import fastapi
import uvicorn

from api import dob_api
from views import home

app = fastapi.FastAPI()


def configure_routing():
    app.include_router(home.router)
    app.include_router(dob_api.router)


if __name__ == '__main__':
    configure_routing()
    uvicorn.run(app, port=8000, host='127.0.0.1')
else:
    configure_routing()

The requirements.txt will contain the packages you used in the API. In this case:

```
fastapi
uvicorn
httpx
aiofiles
requests
uvloop
httptools
BeautifulSoup4
```

Take a moment now to create a folder with the necessary files to make your API run, you can use the same one we created in the last lesson. Make sure that the file containing the FastAPI is called `main.py`. We provided a folder named `example` with the right structure.

5. `cd` to your project folder and type `deta new`. You will see something like this:
<p align=center><img src=images/Deta_new.png width=200></p>

In your case, the endpoint will be different. That is the HTTPS where your API will be deployed. If for some reason, your `http_auth` is enable, you won't be able to publish the API with the rest of the world. Then, you can disable it by running: 

`deta auth disable`

If you find any issue when opening that URL, you can see the error running `deta visor open` and your browser will show you the errors you have

You can take a look at the parameters you can use by going to: [https://x8aqtu.deta.dev/docs](https://x8aqtu.deta.dev/docs):
<p align=center><img src=images/Deta_docs.png width=400></p>

Or simply, create a request:

In [1]:
import requests
requests.get('https://x8aqtu.deta.dev/api/dob/Steve?last_name=Carrel&city=true').json()

{'first name': 'Steve',
 'last_name': 'Carrel',
 'Date of Birth': '1962-08-16',
 'City': 'Concord, Massachusetts, U.S.'}

You saw that you can simply upload your API in a few steps very easily. 

However, this is not as scalable as having your API deployed in an EC2 instance.